In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import os.path
import csv
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from random import randint


class PatientPhenotype:
    
    def __init__(self, eid, case, sex, yearBirth):
        
        self.eid = eid.strip()
        self.case = int (case.strip())
        self.sex = sex.strip()
        self.yearBirth = yearBirth.strip()
        self.snps = {}
        
    def getEid(self):
        return self.eid
     
    def getCase(self):
        return self.case
    
    def getSex(self):
        return self.sex
    
    def getYearBirth(self):
        return self.yearBirth
        
    def addSnps(self, snpId, allele1,allele2):
        self.snps[snpId] = Snp(snpId,allele1,allele2)
        
    def snpCode(self,chromosomes = {}, snp = '', code = -1):
    
        if len(chromosomes.keys()) > 0:
    
             for i in range(len(chromosomes.keys())):
    
                chro = 'chr'+str(i+1)
            
                for snp in chromosomes[chro].keys():
                
                    allele1 = chromosomes[chro][snp][0].strip()
                    allele2 = chromosomes[chro][snp][1].strip()
                    
                    self.snps[snp.strip()].setSnpCode(allele1,allele2)
                    
        else:
            
            self.snps[snp.strip()].setCode(code)
            
    def getSnpCode(self,snpId):
        return self.snps[snpId].getSnpCode()
    
    def getAllele1(self,snpId):
        return self.snps[snpId].getAllele1()
    
    def getAllele2(self,snpId):
        return self.snps[snpId].getAllele2()
        
    def getSize(self):
        return len(self.snps)
        
        
class Snp:
    
    def __init__(self,snpId,allele1,allele2):
        
        self.snpId = snpId
        self.allele1 = allele1
        self.allele2 = allele2
        self.snpCode = -1
        
    def getId(self):
        
        return self.snpId
        
    def getAllele1(self):
        
        return self.allele1
        
    def getAllele2(self):
        
        return self.allele2
        
    def setSnpCode(self,allele1,allele2):
      
        if self.allele1.strip() == allele1.strip() and self.allele2.strip() == allele1.strip():
            code = 2
           
        elif self.allele1.strip() == allele1.strip() and self.allele2.strip() != allele1.strip():
            code = 1
           
        elif self.allele1.strip() != allele1.strip() and self.allele2.strip() == allele1.strip():
            code = 1
            
        elif self.allele1.strip() != allele1.strip() and self.allele2.strip() != allele1.strip():
            code = 0
            
        self.snpCode = code
        
    def setCode(self,aCode):
        
        self.snpCode = aCode
        
    def getSnpCode(self):
        
        return self.snpCode
    
class Read:
    def __init__(self,path,numberOfChromosomes):
        
        self.chromosomes = {}
        self.numberOfSnps = 0
        self.path = path
        self.numberOfChromosomes = numberOfChromosomes
        
    def readPatients(self,kind):
        patients = {}
        f = open(self.path + kind,'r')
        f.readline()

        for line in f:
            patients[line.split()[0].strip()] = PatientPhenotype(line.split()[0],line.split()[3],line.split()[1],line.split()[2])
        
        f.close()
        
        return patients
    
    def readSnps(self,fileKind):
        
        for i in range(self.numberOfChromosomes):
    
            chro = 'chr'+str(i+1)
            path = self.path + chro + fileKind
            f = open(path,'r')
            f.readline()
    
            self.chromosomes[chro] = self.__readSnpsOfChromosome(f)
        
            f.close()
        
        return self.chromosomes
    
    def __readSnpsOfChromosome(self,file):
        
        snps = {} 
       
        for line in file:
            
            alleles = []
            alleles.append(line.split()[3].strip())
            alleles.append(line.split()[6].strip())
            
            if line.split()[1].strip() != '.':
                snps[line.split()[1].strip()] = alleles
                self.numberOfSnps += 1
                
        return snps
        
    def readLgen(self,patients,kind = ''):
        
        
        for i in range(self.numberOfChromosomes):
            
            chro = 'chr'+str(i+1)
            path = self.path + chro + kind +'.lgen'
    
            if os.path.exists(path):
                
                f = open(path,'r')
              
                for line in f:
                    
                    if line.split()[0].strip() in patients.keys():
                        
                        patients[line.split()[0].strip()].addSnps(line.split()[2].strip(),line.split()[3].strip(),
                                                                                              line.split()[4].strip())
                f.close()
       
        return patients
    
    def getListOfSnps(self):
        snps = []
        for i in range(self.numberOfChromosomes):
            chro = 'chr'+str(i+1)
            for snp in self.chromosomes[chro].keys():
                snps.append(snp)
        
        return snps
        
    def getNumberOfSnps(self):
        
        return self.numberOfSnps
    
    def readIds(self,kind):
        
        ids = {}
        
       # ids['patients'] = self.__readPatientsId(kind)
        #ids['snps'] = self.__readSnpsId()
        
        return ids
    
    def __readPatientsId(self,kind):
        
        ids = {}
        idToName = {}
        nameToId = {}
        
        read = open(self.path + kind + 'Ids.txt','r')    
        read.readline()
        
        for line in read:
            
            name = line.split('\t')[0].strip()
            idi = int (line.split('\t')[1].strip())
            
            nameToId[name] = idi
            idToName[idi] = name
            
        ids['nameToId'] = nameToId
        ids['idToName'] = idToName
        
        read.close()
                      
        return ids  
    
    
    def __readSnpsId(self):
        
        ids = {}
        idToName = {}
        nameToId = {}
        
        read = open(self.path + 'SnpsIds.txt','r')    
        read.readline()
        
        for line in read:
            
            name = line.split('\t')[0].strip()
            idi = int (line.split('\t')[1].strip())
            
            nameToId[name] = idi
            idToName[idi] = name 
                      
        ids['nameToId'] = nameToId
        ids['idToName'] = idToName

        read.close()
                      
        return ids  
    
    def readSnpsCode(self,patients,kind = ''):
        
        read = open(self.path + kind + 'snpCode.txt','r')
        read.readline()
        read.readline()
        print("mphka2")
        for line in read:   
        
            patient = line.split('\t')[0].strip()
            snp = line.split('\t')[1].strip()
            code = int (line.split('\t')[2].strip())
            allele1 = line.split('\t')[3].strip()
            allele2 = line.split('\t')[4].strip()
            if patient in patients.keys() and snp != '.':
                patients[patient].addSnps(snp,allele1,allele2)
                patients[patient].snpCode(snp = snp,code = code)
         
        read.close()
            
        return patients
            
        
    
class Write:
    
    def __init__(self,path,numberOfChromosomes):
        
        self.path = path
        self.numberOfChromosomes = numberOfChromosomes
        
    def writePatientsList(self,patients,kind):
        
        path = self.path + kind
        write = open(path,'w')
        for patient in patients.keys():
            write.write(patient.strip() + '\n')
            
        write.close()
        
        
    def writeSnpsList(self,chromosomes):
        
        for i in range(self.numberOfChromosomes):
    
            chro = 'chr'+str(i+1)
            path = self.path + chro + 'snpList.txt'
            write = open(path,'w')
            
            for snp in chromosomes[chro].keys():
                write.write(snp.strip() + '\n')
            
            write.close()
            
    def saveData(self,ids,patients,patientKind,data,chroms = {}):
        
        #self.__patientsLogFile(ids['patients'],patientKind)
        #self.__snpsLogFile(ids['snps'],chroms)
        self.__snpCodeLog(ids['patients']['idToName'],ids['snps']['idToName'],patients,data,patientKind)
        
    def __patientsLogFile(self,ids,patientKind):
        
        write = open(self.path + patientKind + 'Ids.txt','w')
        
        write.write(str(len(ids['nameToId'])) + '\n')
        
        for patient in ids['nameToId'].keys():
            
            write.write(patient.strip() + '\t' + str(ids['nameToId'][patient]).strip() + '\n')
            
        write.close()
        
    def __snpsLogFile(self,ids,chroms):
        
        if len(chroms.keys()) > 0:
        
            write = open(self.path + 'SnpsIds.txt','w')
        
            write.write(str(len(ids['nameToId'])) + '\n')
        
            for chro in chroms.keys():
              
                for snp in chroms[chro].keys():
                    write.write(snp.strip() + '\t' + str(ids['nameToId'][snp.strip()]).strip() + '\n')
            
            write.close()
            
    def __snpCodeLog(self,patientsIds,snpsIds,patients,data,patientKind):
        
        write = open(self.path + patientKind + 'snpCode.txt','w')
        
        write.write(str(len(patientsIds)) + '\n')
        write.write(str(len(snpsIds)) + '\n')
        
        for i in range(len(data)):
            for j in range(len(data.T)):
                allele1 = patients[patientsIds[i]].getAllele1(snpsIds[j])
                allele2 = patients[patientsIds[i]].getAllele2(snpsIds[j])
                write.write(patientsIds[i].strip() + '\t' + snpsIds[j].strip() + '\t' + str(data[i,j]).strip() + '\t' 
                                                                            + allele1.strip() + '\t' + allele2.strip() + '\n')
                
        write.close()
        
        
            
class DataSet:
    
    def __init__(self,patients,ids):
        
        self.n = len(ids['patients']['nameToId'].keys())
        self.m =len(ids['snps']['nameToId'].keys()) 
        self.patients = patients
        self.ids = ids
                     
        self.xTable = np.zeros((self.n,self.m),dtype = int)
        self.yTable = np.zeros((self.n,),dtype = int)
        
        for i in range(self.n):
            for j in range(self.m):
                self.xTable[i,j] = -1
                     
        self.__fillXTable()
        self.__fillYTable()
                     
                     
    def __fillXTable(self):
    
        for i in range(self.n):
            for j in range(self.m):
        
                patient = self.ids['patients']['idToName'][i]
                snp = self.ids['snps']['idToName'][j]
        
                self.xTable[i,j] = self.patients[patient].getSnpCode(snp)
                     
    def __fillYTable(self):
    
        for i in range(self.n):
    
            patient = self.ids['patients']['idToName'][i]
            self.yTable[i] = self.patients[patient].getCase()
        
    def getXTable(self):
                     
        return self.xTable
                     
    def getYTable(self):
                     
        return self.yTable
    

def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    return ids


def setSnpsCode(patients,chromosomes):
    
    for i in patients.keys():
        patients[i].snpCode(chromosomes)
        
    return patients
    
    
def mergeXTrainXTestTable(test,train):
    
    n = len(test) + len(train)
    m = len(test.T)
    count = 0
    
    mergeTable = np.zeros((n,m),dtype = int)
    
    for i in range(len(train)):
        for j in range(len(train.T)):
            mergeTable[count,j] = train[i,j]
        count += 1
    
    
    for i in range(len(test)):
        for j in range(len(test.T)):
            mergeTable[count,j] = test[i,j]
        count += 1

        
    return mergeTable


def mergeYTrainYTestTable(test,train):
    
    n = len(test) + len(train)
   
    count = 0
    
    mergeTable = np.zeros((n,),dtype = int)
    
    for i in range(len(train)):
        mergeTable[count] = train[i]
        count += 1
    
    for i in range(len(test)):
        mergeTable[count] = test[i]
        count += 1
        
    return mergeTable

def createAllPatientsStructure(patients,patientsTest,patientsTrain,snps):
    
    for i in patientsTest.keys():
        patients[i]=patientsTest[i]
        
    for i in patientsTrain.keys():
        patients[i]=patientsTrain[i]
        
    return patients

def seperatePatients(patients,allPatients):
    
    for patient in patients.keys():
        
        patients[patient] = allPatients[patient]
        
    return patients


def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_


def tables(sampleX,sampleY,k):
  
    samples = {}
    
    for run in range(1,k+1):
        
        d1 = {}
        

        dataTestX = sampleX[run]
        dataTestY = sampleY[run]

        n = 0

        for i in sampleX.keys():

            if i != run:

                n += len(sampleX[i])

        dataTrainX = np.zeros((n,len(sampleX[1].T)),dtype = int)
        dataTrainY = np.zeros((n,),dtype = int)

        count = 0

        for sample in sampleX.keys():

            if sample != run:

                 for i in range(len(sampleX[sample])):
                    for j in range(len(sampleX[sample].T)):
                        dataTrainX[count,j] = sampleX[sample][i,j]

                    dataTrainY[count] = sampleY[sample][i]
                    count += 1

        d1['trainX'] = dataTrainX
        d1['trainY'] = dataTrainY
        d1['testX'] = dataTestX
        d1['testY'] = dataTestY
        
        samples[run] = d1
    
    return samples
    
def kSampleData(k,X,Y):
    
    x = int (len(X) / k)
    allElements = np.zeros((len(X),),dtype = int)
    
    count1 = 1
    sampleX = {}
    sampleY = {}
   
    
    while count1 <= k:
        count2 = 1
        sampleData = []
        
        if count1 == k:
            x =  len(X) - ((k-1) * x)
        
        dataX = np.zeros((x,len(X.T)),dtype = int)
        dataY = np.zeros((x,),dtype = int)
        
        while count2 <= x:
            
            aRand = randint(0,len(X)-1)
            
            while allElements[aRand] == 1:
                
                aRand = randint(0,len(X)-1)
            
            allElements[aRand] = 1
            sampleData.append(aRand)
            count2 += 1
            
        for i in range(len(sampleData)):
            for j in range(len(X.T)):
                dataX[i,j] = X[sampleData[i],j]
            
            dataY[i] = Y[sampleData[i]]
            
        sampleX[count1] = dataX
        sampleY[count1] = dataY
        count1 +=1
        
    return tables(sampleX,sampleY,k)

def crossValidiation(X, Y, k = 1, continious = True, classifier = None,OLS = False):
    
    sumResults = 0
    
    if not classifier:
        print("wrong!!!!!!! you have to choise a classifier")
        return
    
    results = {}
    samples = kSampleData(k,X,Y)
    
    for run in range(1, k + 1):
        
        trainX = samples[run]['trainX']
        trainY = samples[run]['trainY']
        
        testX = samples[run]['testX']
        testY = samples[run]['testY']
        
        if OLS:
            classifier = sm.OLS(trainY,trainX)
            yPredict = classifier.fit().predict(testX)
        else:
            classifier.fit(trainX, trainY)
            yPredict = classifier.predict(testX)
        
        if continious:
            
            count = 0
            
            for i in range(len(yPredict)):
                count += yPredict[i]
                
            mo = count / len(yPredict)
            
            for i in range(len(yPredict)):
                
               # if yPredict[i] < mo:
                if abs(0 - yPredict[i]) < abs(1 - yPredict[i]):
                    yPredict[i] = 0
                else:
                    yPredict[i] = 1
                    
        results[run] = metrics.accuracy_score(yPredict,testY)
    
    for i in results.keys():
        
        sumResults = sumResults + results[i]
    
    mo = sumResults / k
    
    return mo
           

    
def reduceFeatures(X,cosineSimilarity = False):
    
    snps = []
    threshold = 55
    
    if cosineSimilarity:
        
        snpsCount = {}
        print("size = ",len(X.T))
        for i in range(len(X.T)):
            
            snpsCount[i] = 0
        
        xNew = metrics.pairwise.cosine_similarity(X.T)
        
        '''count1 = 0
        count2 = 0
        count3 = 0
        for i in range(len(xNew)):
            for j in range(i+1,len(xNew.T)):
                
              
            #   if float("%.2f" % (1 - xNew[i,j])) <= 1.0 ) and float("%.2f" % (1 - xNew[i,j])) >= 0.0) :
                
                if float("%.2f" % xNew[i,j]) <= 1.0 and float("%.2f" % xNew[i,j]) >= 0.0:
                    count1 += 1
                    #print("xNew = ",xNew[i,j])
                    
             #   else:
                   # if count3 <= 4:
              #          print("i = ",i)
               #         print("j = ",j)
               #         print("xNew = ",1 - xNew[i,j])
               #         count3 += 1
                
                   
                    
                count2 += 1
        
        print("count1 = ",count1)
        print("count2 = ",count2)'''
        
        print("xNew shape ",xNew.shape)
        
        for i in range(len(xNew)):
            for j in range(i+1,len(xNew.T)):
                if float("%.2f" % xNew[i,j]) <= 0.2:
                    snpsCount[j] = snpsCount[j] + 1
        count = 0        
        for i in snpsCount.keys():
            if snpsCount[i] >= 50 * len(X.T) / 100:
                count += 1
                snps.append(i)
         #   print(snpsCount[i])   
        print("count = ",count)
        print("len snps = ",len(snps))
        
    else:
    
        for i in range(len(X.T)):

            count0 = 0
            count1 = 0
            count2 = 0 

            for j in range(len(X)):

                if X[j,i] == 0:
                    count0 += 1

                elif X[j,i] == 1:
                    count1 += 1

                elif X[j,i] == 2:
                    count2 +=1

          #  if count0 < thresold *len(X)/100 and count1 < thresold *len(X)/100 and count2 < thresold *len(X)/100:
            if count0 != 0 and count1 !=0 and count2 !=0:
                snps.append(i)
            
    return snps


def createNewTable(snps,X):
    
    newX = np.zeros((len(X),len(snps)),dtype = int)
    
    for i in range(len(newX)):
        for j in range(len(newX.T)):
            newX[i,j] = -1
    
    for i in range(len(snps)):
        for j in range(len(X)):
            
            newX[j,i] = X[j,snps[i]]
            
    print("new shape = ",newX.shape)
            
    return newX
    

In [ ]:
path = 'C:\\Users\\ANTONIS\\Desktop\\p = 0.0001\\' #bake ton fakelo pou 8a epe3ergas8eis 
numberOfChromosomes = 22#'ari8mos twn xromoswmatwn'
patientsTrain = {}
patientsTest = {}
allPatients = {}

chromosomes = {}

read = Read(path,numberOfChromosomes)
write = Write(path,numberOfChromosomes)

patientsTrain = read.readPatients('phenotype_euro_balanced_train9.txt')
patientsTest = read.readPatients('phenotype_euro_balanced_test9.txt')

chromosomes = read.readSnps(".assoc")

write.writePatientsList(patientsTrain,'trainPatient.txt')
write.writePatientsList(patientsTest,'testPatient.txt')
write.writeSnpsList(chromosomes)


# run train_lgen bat and test_leg bat

In [ ]:

snps = read.getListOfSnps()
ids = {} 
idsTest = {}

allPatients = createAllPatientsStructure(allPatients,patientsTest,patientsTrain,snps)

if os.path.exists(path + 'snpCode.txt'):
    print("mphka")
    allPatients = read.readSnpsCode(allPatients)
    
    patientsTrain = seperatePatients(patientsTrain,allPatients)
    patientsTest = seperatePatients(patientsTest,allPatients)
    
else:
    allPatients = read.readLgen(allPatients)
    
    patientsTrain = seperatePatients(patientsTrain,allPatients)
    patientsTest = seperatePatients(patientsTest,allPatients)
    
    patientsTrain = setSnpsCode(patientsTrain,chromosomes)
    patientsTest = setSnpsCode(patientsTest,chromosomes)
    


ids['patients'] = setIdToName(list(patientsTrain.keys()))
ids['snps'] = setIdToName(snps)

idsTest['patients'] = setIdToName(list(patientsTest.keys()))
idsTest['snps'] = ids['snps']




In [ ]:
trainSet = DataSet(patientsTrain,ids)
testSet = DataSet(patientsTest,idsTest)

xTraining = trainSet.getXTable()
yTraining = trainSet.getYTable()
        
xTest = testSet.getXTable()
yTest = testSet.getYTable()


mergeXtable = mergeXTrainXTestTable(xTraining,xTest)
mergeYtable = mergeYTrainYTestTable(yTraining,yTest)
  
if not os.path.exists(path + 'snpCode.txt'):
    print("mphka 3")
    write.saveData(ids,patientsTrain,'Train',xTraining,chromosomes)
    write.saveData(idsTest,patientsTest,'Test',xTest)

In [ ]:
snpsReduced = reduceFeatures(xTraining,cosineSimilarity = True)
mergeXtable = createNewTable(snpsReduced,mergeXtable)
xTraining = createNewTable(snpsReduced,xTraining)
xTest = createNewTable(snpsReduced,xTest)

In [ ]:
count = 0

for patient in patientsTrain.keys():
            
    if patientsTrain[patient].getSize() == read.getNumberOfSnps():
        
        count += 1
        
print("countTrain is ",count)
print("patientsTrain is ",len(patientsTrain.keys())) 
print("snps is Train",read.getNumberOfSnps())
print("shape id ",xTraining.shape)
print("xTraining = ",len(xTraining))
print("xTraining.T = ",len(xTraining.T))
print("yTraining = ",len(yTraining))

print()

count = 0

for patient in patientsTest.keys():
            
    if patientsTest[patient].getSize() == read.getNumberOfSnps():
        
        count += 1
        
print("count is Test",count)
print("patientsTest is ",len(patientsTrain.keys()))
print("snps is Test",read.getNumberOfSnps())
print("shape id Test",xTest.shape)
print("xtest = ",len(xTest))
print("xtest.T = ",len(xTest.T))
print("ytest = ",len(yTest))

print()
print("mergeXtable = ",len(mergeXtable))
print("mergeXtable.T = ",len(mergeXtable.T))
print("mergeYtable = ",len(mergeYtable))
print("shape id ",mergeXtable.shape)

print()


print("id to name patients",len(ids['patients']['idToName'].keys()))
print("name to id patients",len(ids['patients']['nameToId'].keys()))
print()
print("id to name snps",len(ids['snps']['idToName'].keys()))
print("name to id snps",len(ids['snps']['nameToId'].keys()))

print()
count0 = 0
count1 = 0
count2 = 0
count = 0

for i in range(len(xTraining)):
    for j in range(len(xTraining.T)):
        if xTraining[i,j] == 2:
            count2 += 1
                
        elif xTraining[i,j] == 1:
            count1 += 1
                
        elif xTraining[i,j] == 0:
            
            count0 +=1
        else:
            count += 1
            
print("xTraining")
print("count = ",count)
print("count0 = ",count0)
print("count1 = ",count1)
print("coun2 = ",count2)

print()

count0 = 0
count1 = 0
count2 = 0
count = 0

for i in range(len(xTest)):
    for j in range(len(xTest.T)):
        if xTest[i,j] == 2:
            count2 += 1
                
        elif xTest[i,j] == 1:
            count1 += 1
                
        elif xTest[i,j] == 0:
            
            count0 +=1
        else:
            count += 1
            
print("xTest")
print("count = ",count)
print("count0 = ",count0)
print("count1 = ",count1)
print("coun2 = ",count2)

count0 = 0
count1 = 0
count2 = 0
count = 0

for i in range(len(mergeXtable)):
    for j in range(len(mergeXtable.T)):
        if mergeXtable[i,j] == 2:
            count2 += 1
                
        elif mergeXtable[i,j] == 1:
            count1 += 1
                
        elif mergeXtable[i,j] == 0:
            
            count0 +=1
        else:
            count += 1
print()
            
print("mergeXtable")
print("count = ",count)
print("count0 = ",count0)
print("count1 = ",count1)
print("coun2 = ",count2)

# # RF

In [ ]:
print(xTraining.shape)
print(xTest.shape)
print(mergeXtable.shape)

In [ ]:
rfr = RandomForestRegressor(n_estimators = 500, random_state = 2016, verbose = 20,max_depth = None,n_jobs=2)
rfr.fit(xTraining, yTraining)
xtraining = rfr.transform(xTraining)
xtest = rfr.transform(xTest)
mergex = rfr.transform(mergeXtable)

In [ ]:
print(xtraining.shape)
print(xtest.shape)
print(mergex.shape)

In [ ]:
rfr = RandomForestRegressor(n_estimators = 500, random_state = 2017, verbose = 20,
                            max_depth = None,n_jobs=2)
rfr.fit(xTraining, yTraining)
yPredict3 = rfr.predict(xTest)

count = 0
for i in range(len(yPredict3)):
    count += yPredict3[i]

mo3 = count / len(yPredict3)

for i in range(len(yPredict3)):
    #if yPredict3[i] < mo3:
    if abs(0 - yPredict3[i]) < abs(1- yPredict3[i]):
        yPredict3[i] = 0
    else:
        yPredict3[i] = 1
        
print(metrics.accuracy_score(yTest,yPredict3))
print(metrics.confusion_matrix(yTest,yPredict3))
error3 = mean_squared_error(yTest, yPredict3)
print("error 3 = ",error3)
RMSE3 = mean_squared_error(yTest,yPredict3)**0.5
print("RMSE3 = ",RMSE3)

print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = rfr))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict3)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict3))

# Linear Regression

In [ ]:


regr = linear_model.LinearRegression()
regr.fit(xTraining, yTraining)
yPredict1 = regr.predict(xTest)



In [ ]:
count = 0

for i in range(len(yPredict1)):
    count += yPredict1[i]

mo = count / len(yPredict1)

for i in range(len(yPredict1)):
    if abs(0 - yPredict1[i]) < abs(1-yPredict1[i]):
        yPredict1[i] = 0
    else:
        yPredict1[i] = 1

error1 = mean_squared_error(yTest, yPredict1)
print("error 1 = ",error1)
print(metrics.accuracy_score(yTest,yPredict1))
print(metrics.confusion_matrix(yTest,yPredict1))
RMSE1 = mean_squared_error(yTest,yPredict1)**0.5
print("RMSE1 = ",RMSE1)


print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = regr))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict1)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict1))

# SVM

In [ ]:

clf = SVC(kernel='linear')
clf.fit(xTraining, yTraining)
yPredict2 = clf.predict(xTest)
print(metrics.accuracy_score(yTest,yPredict2))
print(metrics.confusion_matrix(yTest,yPredict2))
error2 = mean_squared_error(yTest, yPredict2)
print("error 2 = ",error2)
RMSE2 = mean_squared_error(yTest,yPredict2)**0.5
print("RMSE2 = ",RMSE2)

print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = clf))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict2)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict2))

# LINEAR LOGISTIC REGRESSION 

In [ ]:


lr_clf = linear_model.LogisticRegression()  
                                 
    

#(multi_class ='multinomial',solver = 'sag',cv = 10,penalty ='l2',
 #                                          class_weight='balanced')

#(penalty='l2', alpha=0.000000000001, fit_intercept=False,
 #               n_iter=100, shuffle=True, verbose=2016, eta0=0.0000000001, n_jobs=-1, random_state=2016, warm_start=False)
                                 
                                 #(penalty='l1', alpha=0.00000001, fit_intercept=True,
        #       n_iter=100, shuffle=True, verbose=2016, eta0=0.00000001, n_jobs=-1, random_state=2016, warm_start=True)
    
#LogisticRegression()  
    
lr_clf.fit(xTraining, yTraining)
yPredict4 = lr_clf.predict(xTest)


'''count = 0
for i in range(len(yPredict4)):
    count += yPredict4[i]
mo = count / len(yPredict4)

for i in range(len(yPredict4)):
    if yPredict4[i] < mo:
        yPredict4[i] = 0
    else:
        yPredict4[i] = 1
'''

print(metrics.accuracy_score(yTest,yPredict4))
print(metrics.confusion_matrix(yTest,yPredict4))
error4 = mean_squared_error(yTest, yPredict4)
print("error 4 = ",error4)
RMSE4 = mean_squared_error(yTest,yPredict4)**0.5
print("RMSE4 = ",RMSE4)

print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = lr_clf))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict4)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict4))

# OLS

In [ ]:
ols = sm.OLS(yTraining,xTraining)#OLS(yTraining,xtraining)
yPredict5 = ols.fit().predict(xTest)

mo5 = 0
s5 = 0
for i in yPredict5:
    s5 += i
    
mo5 = s5 / len(yPredict5)

In [ ]:

for i in range(len(yPredict5)):
    
    #if abs(0 - yPredict5[i]) < abs(1 - yPredict5[i]):
    
    if yPredict5[i] <mo5:
        yPredict5[i]=0
    else:
        yPredict5[i]=1

print(metrics.accuracy_score(yTest,yPredict5))
print(metrics.confusion_matrix(yTest,yPredict5))
error5 = mean_squared_error(yTest, yPredict5)
print("error 5 = ",error5)
RMSE5 = mean_squared_error(yTest,yPredict5)**0.5
print("RMSE5 = ",RMSE5)

print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = ols,OLS = True))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict5)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict5))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xTraining, yTraining)
yPredict6 = gnb.predict(xTest)
print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)

print("cros validation = ",crossValidiation(mergeXtable, mergeYtable, k = 10, classifier = gnb))
fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict6))

In [ ]:
'''results = crossValidiation(mergex, mergeYtable, k = 10, classifier = gnb)
results2 = cross_val_score(gnb, mergex, mergeYtable, cv=10)
count1 = 0
count2 = 0
for i in results2:
    count2 += i
    
for i in results.keys():
    count1 += results[i]
    
print("mo1 = ",count1/10)
print("mo2 = ",count2/10)
'''

In [ ]:
 
'''RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

parameters = {}
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
#parameters = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
    
gsv = GridSearchCV(estimator = rfr, param_grid = parameters, cv = cv, verbose = 2016)#, scoring=RMSE)
gsv.fit(xTraining, yTraining)

yPredict6 = gsv.predict(xTest)'''


In [ ]:
'''count = 0
for i in range(len(yPredict6)):
    count += yPredict6[i]

mo6 = count / len(yPredict6)

for i in range(len(yPredict6)):
    if yPredict6[i] < 0.5:
        yPredict6[i] = 0
    else:
        yPredict6[i] = 1

print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)'''